In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [48]:
pip show PINECO

Note: you may need to restart the kernel to use updated packages.


### Chunking Data

In [18]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks
    

### Embedding and Uploading to a Vector Database (Pinecone)

In [21]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store
    

In [22]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Ok')
    

### Asking and Getting Answers

In [23]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    return answer
    
    
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history
    

### Loading the Document

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('ai-in-india.pdf')
data = loader.load()

You have 52 pages in your data
There are 1151 characters in the page


In [8]:
data

[Document(page_content='AI in India -  \nA Strategic Necessity\nA pragmatic playbook for Indian \norganizations to leapfrog on AI maturity \nJuly 2023\n', metadata={'source': 'ai-in-india.pdf', 'page': 0}),
 Document(page_content='Boston Consulting Group partners with leaders \nin business and society to tackle their most \nimportant challenges and capture their greatest \nopportunities. BCG was the pioneer in business \nstrategy when it was founded in 1963. Today, \nwe work closely with clients to embrace a \ntransformational approach aimed at benefiting all \nstakeholders—empowering organizations to grow, \nbuild sustainable competitive advantage, and \ndrive positive societal impact.\nOur diverse, global teams bring deep industry and \nfunctional expertise and a range of perspectives \nthat question the status quo and spark change. \nBCG delivers solutions through leading-edge \nmanagement consulting, technology and design, \nand corporate and digital ventures. We work in a \nunique

In [9]:
len(data)  #Number of pages

52

### Chunking Data

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
chunks = text_splitter.split_documents(data)

In [28]:
print(len(chunks))

247


In [ ]:
#Cost Estimate

In [29]:
import tiktoken
enc = tiktoken.encoding_for_model('text-embedding-ada-002')
total_tokens = sum([len(enc.encode(page.page_content)) for page in chunks])
print(f'Total Tokens: {total_tokens}')
print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

Total Tokens: 25223
Embedding Cost in USD: 0.010089


In [36]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [37]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...Ok


In [38]:
q1 = 'What is the whole document about?'
answer = ask_and_get_answer(vector_store, q1)
print(answer)

The document appears to cover various topics related to AI (Artificial Intelligence) including global perspectives on AI, AI maturity survey, strategic planning in the era of AI, adoption process of analytics in organizations, responsible AI, India's AI policy, and the future of work in India. It may also touch upon topics such as automation, the future of retail banking, and technology platforms.


In [42]:
q2 = 'India''s success in AI will be shaped by what?'
answer = ask_and_get_answer(vector_store, q2)
print(answer)

India's success in AI will be shaped by four key factors: 
1) Nurturing AI talent: India needs to increase its pool of AI professionals and ensure they have the necessary business and sector understanding.
2) Direct hire of AI talent: Companies in India are struggling to find AI talent with the required skills, leading to a talent crunch.
3) Training and skills development: India needs to invest in training programs to develop AI skills, including management, design thinking, domain knowledge, Agile working, and management of digital organizations.
4) AI investments and economic growth: Successful adoption of AI has the potential to add up to 1.4 percentage points annually to India's real GDP growth and drive economic expansion.


In [43]:
q3 = 'What exactly is AI Maturity?'
answer = ask_and_get_answer(vector_store, q3)
print(answer)

AI Maturity refers to the overall capability of a company to adopt and integrate artificial intelligence (AI) into its operations and strategic objectives. It measures how effectively an organization is able to leverage AI to drive its financial and operational performance. AI maturity goes beyond just having analytical or technical capabilities and encompasses the ability to holistically embrace AI to gain a competitive advantage. Companies with higher AI maturity are typically better positioned to serve and retain customers, innovate consistently, and maintain a competitive edge in the market. Conversely, companies with low AI competency are at risk of losing market share and margins to competitors with superior AI capabilities.


In [44]:
q4 = 'How is BFSI performing in AI?'
answer = ask_and_get_answer(vector_store, q4)
print(answer)

The BFSI industry is performing well in terms of AI maturity. Leaders in BFSI have better operating metrics compared to the rest of the industry. They have a solid credit growth of 3 percentage points ahead of other groups. Three years ago, the sector had a score of 3.2 in AI maturity, which has improved to 3.9. The industry's AI leadership can be attributed to its long association with data-driven decision making and the investments made in technology, organization, people, and procedures. Some Indian BFSI companies are on par with global frontrunners in terms of AI maturity.


In [45]:
q5 = 'What is the scope of AI for Consumer Goods?'
answer = ask_and_get_answer(vector_store, q5)
print(answer)

The scope of AI for Consumer Goods is quite broad. It can be applied in various areas such as demand forecasting, supply chain optimization, quality assurance, customer behavior analysis, sentiment analysis, and product review analysis. AI can help Consumer Goods companies in improving their brand and innovation strategies, winning in the marketplace, and meeting compliance standards. However, it is important to note that the Consumer Goods industry currently has limited AI maturity, with only a small percentage of organizations being leaders in AI adoption.


In [46]:
q6 = 'What are the challenges for AI in Industrial Goods?'
answer = ask_and_get_answer(vector_store, q6)
print(answer)

The challenges for AI in Industrial Goods include:

1. Supply chain issues: AI can help optimize supply chain processes, but there may be challenges in implementing AI-powered demand forecasting at different levels such as SKU, geography, and channel.

2. Compliance to quality standards: Ensuring compliance with global quality standards and Environmental, Social, and Governance (ESG) requirements can be a challenge when adopting AI in industrial goods.

3. Security: Maintaining data security and privacy while implementing AI solutions in the industrial goods sector is crucial to prevent unauthorized access or data breaches.

4. Quality assurance: Implementing AI for quality assurance purposes, such as detecting defects or ensuring product quality, may require overcoming technical and operational challenges.

5. Understanding customer behavior: Developing a deep understanding of customer behavior and preferences in order to carve a niche and win market volume can be a challenge, especia

In [47]:
q7 = 'What are the key questions of Responsible AI?'
answer = ask_and_get_answer(vector_store, q7)
print(answer)

The key questions regarding Responsible AI include:

1. Ownership and authenticity: Who owns the generated content? How can we ensure the authenticity of the content created by AI?

2. Misuse and manipulation: What are the potential risks and implications of AI being misused or manipulated?

3. Impact on employment and creativity: How will the use of AI impact job opportunities and the creative industries?

4. Ethical considerations: What ethical guidelines should be followed in the development and implementation of AI?

5. Inclusion of experts: Are AI ethics experts involved in strategic discussions and decision-making processes?

6. Understanding of guidelines: Are the guidelines and procedures for Responsible AI well understood throughout the organization?

7. Privacy violations: How can we prevent data breaches and protect sensitive information in AI systems?

8. Unintended side-effects: How can we anticipate and address unintended consequences and side-effects of AI systems?

It i

In [40]:
q8 = 'What should the strategy be for India'
answer = ask_and_get_answer(vector_store, q8)
print(answer)

Based on the provided context, the strategy for India in leveraging AI should focus on nurturing AI talent, acquiring the necessary business and sector understanding, and developing an end-to-end analytics process. It is essential to address the talent crunch by investing in the development of AI professionals and encouraging their direct hiring by companies. Additionally, India should focus on planning early for industrialization and developing a comprehensive analytics process to ensure successful adoption of AI.


In [43]:
delete_pinecone_index()

Deleting all indexes ... 
Ok
